# Voxelisation

To perform a multi scale completion, we need data that can be completed at different scales, while being spacially consistent for the machine learning models.
This is why we use voxel hierarchies, combined with TSDFs to generate the scene at multiple levels of detail.

## Open3D

Using open3d we first sample the geometry to a uncolored voxelgrid, where the voxelsize is matched to a certain octree size.
The centerpoints of each voxel are then used to determine the color and distance to the mesh.

In [1]:
%load_ext autoreload
%autoreload 2
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
meshPath = r"L:\Recordings\2023-11 Scannetpp\data\0a7cc12c0e\scans\mesh_aligned_0.05.ply"
semanticMeshPath = r"L:\Recordings\2023-11 Scannetpp\data\0a7cc12c0e\scans\mesh_aligned_0.05_semantic.ply"


In [2]:
meshPath = r"C:\Users\jelle\Documents\DoctoraatLocal\generationtools\data\textured_table\scene.gltf"

In [27]:
from context import generationtools as gnt

octreeDepth = 8
print('input')
mesh = o3d.io.read_triangle_mesh(meshPath, True)
# fit to unit cube
mesh.scale(1 / np.max(mesh.get_max_bound() - mesh.get_min_bound()), center=mesh.get_center())
o3d.visualization.draw_geometries([mesh])

input


In [30]:

print('voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size=1/(2**octreeDepth))
voxelPoints = []
voxelList = voxel_grid.get_voxels()
for voxel in voxelList:
    voxelCenterCoord = voxel_grid.get_voxel_center_coordinate(voxel.grid_index)
    voxelPoints.append(voxelCenterCoord)

colors, distances = gnt.get_point_pixel_colors_open3d(mesh,voxelPoints, getDistance=True)
print("colors sampled")



voxelization
Closest points computed
colors sampled


In [33]:
class InfoVoxel(o3d.geometry.Voxel):
    
    distance = 0
    segmentationIndex = 0

    def __init__ (self,distance: float = 0, segmentationIndex:int = 0, **kwargs):
        super().__init__(**kwargs)
        self.distance = distance
        self.segmentationIndex = segmentationIndex

In [34]:
i = 0
for voxel in voxelList:
    newVoxel = InfoVoxel(grid_index = voxel.grid_index, color = colors[i] / 256.0, distance = distances)
    voxel_grid.remove_voxel(voxel.grid_index)
    voxel_grid.add_voxel(newVoxel)
    i+=1

o3d.visualization.draw_geometries([voxel_grid])

In [25]:
voxelOctree = voxel_grid.to_octree(octreeDepth)

o3d.visualization.draw_geometries([voxelOctree])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [42]:
if(mesh.has_triangle_material_ids()):
    print(mesh.triangle_material_ids)
    print(mesh.triangles)

IntVector[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
std::vector<Eigen::Vector3i> with 208 elements.
Use numpy.asarray() to access data.


In [ ]:
print('input')
N = 1000000


mesh = o3d.io.read_triangle_mesh(meshPath)
pcd = mesh.sample_points_uniformly(N)
# fit to unit cube
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
          center=pcd.get_center())
#pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
o3d.visualization.draw_geometries([pcd])


In [ ]:

print('octree division')
octree = o3d.geometry.Octree(max_depth=8)
octree.convert_from_point_cloud(pcd, size_expand=0.01)
o3d.visualization.draw_geometries([octree])

In [ ]:
octreenode = octree.locate_leaf_node(pcd.points[0])

print(octreenode[0])

In [ ]:
def f_traverse(node, node_info):
    early_stop = False

    if isinstance(node, o3d.geometry.OctreeInternalNode):
        if isinstance(node, o3d.geometry.OctreeInternalPointNode):
            n = 0
            for child in node.children:
                if child is not None:
                    n += 1
            print(
                "{}{}: Internal node at depth {} has {} children and {} points ({})"
                .format('    ' * node_info.depth,
                        node_info.child_index, node_info.depth, n,
                        len(node.indices), node_info.origin))

            # we only want to process nodes / spatial regions with enough points
            early_stop = len(node.indices) < 250
    elif isinstance(node, o3d.geometry.OctreeLeafNode):
        if isinstance(node, o3d.geometry.OctreePointColorLeafNode):
            print("{}{}: Leaf node at depth {} has {} points with origin {}".
                  format('    ' * node_info.depth, node_info.child_index,
                         node_info.depth, len(node.indices), node_info.origin))
    else:
        raise NotImplementedError('Node type not recognized!')

    # early stopping: if True, traversal of children of the current node will be skipped
    return early_stop

In [ ]:
octree.traverse(f_traverse)

### Trimesh

In [ ]:
import trimesh
import numpy as np
import generationtools as gnt

mesh = trimesh.load(meshPath)
scale = 1 / np.max(mesh.extents)
center = mesh.centroid
for3matx = np.hstack((np.identity(3) * scale, center.reshape((3,1)) ))
transformMtx = np.vstack((for3matx, [0,0,0,1]))
print(transformMtx)
mesh.apply_transform(transformMtx)

In [ ]:
voxelSize = 64
angel_voxel = mesh.voxelized(1/(voxelSize-1)).hollow()
print(angel_voxel.shape)

In [ ]:
voxelPoints = angel_voxel.points
voxelColors = gnt.get_point_colors_trimesh(mesh, voxelPoints)

In [ ]:
ids = angel_voxel.points_to_indices(voxelPoints)

# We initialize a array of zeros of size X,Y,Z,4 to contain the colors for each voxel of the voxelized mesh in the grid
cube_color=np.zeros([angel_voxel.shape[0],angel_voxel.shape[1],angel_voxel.shape[2],4])

for i in range(len(voxelPoints)):
    cube_color[ids[i][0],ids[i][1], ids[i][2],:] = voxelColors[i]

In [ ]:
# generate a voxelized mesh from the voxel grid representation, using the calculated colors 
voxelized_mesh = angel_voxel.as_boxes(colors=cube_color, )

# Initialize a scene
s = trimesh.Scene()
# Add the voxelized mesh to the scene. If want to also show the intial mesh uncomment the second line and change the alpha channel of in the loop to something <100
s.add_geometry(voxelized_mesh)
# s.add_geometry(mesh)
s.show()

### Barycentric coordinates

In [ ]:
from context import generationtools as gnt
import numpy as np
p = np.array((1,1,1))
a = np.array((0,0,1))
b = np.array((3,0,1))
c = np.array((0,10,1))
u,v,w = gnt.carthesian_to_barycentric(p,a ,b ,c )
print(u,v,w)

In [ ]:
gnt.barycentric_to_carthesian(a,b,c, u, v, w)